# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fc58c19a430>
├── 'a' --> tensor([[ 1.0058,  0.4760, -0.0354],
│                   [ 0.6147,  1.4466, -0.1422]])
└── 'x' --> <FastTreeValue 0x7fc58c19aaf0>
    └── 'c' --> tensor([[-0.5109,  0.0742, -0.8903, -0.1564],
                        [-0.9847, -0.6569, -0.1467,  1.2373],
                        [ 0.1478, -0.4505,  0.1463,  0.1611]])

In [4]:
t.a

tensor([[ 1.0058,  0.4760, -0.0354],
        [ 0.6147,  1.4466, -0.1422]])

In [5]:
%timeit t.a

76.6 ns ± 0.175 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fc58c19a430>
├── 'a' --> tensor([[ 0.4152,  1.7778,  0.4929],
│                   [ 0.3384, -1.1128,  0.2333]])
└── 'x' --> <FastTreeValue 0x7fc58c19aaf0>
    └── 'c' --> tensor([[-0.5109,  0.0742, -0.8903, -0.1564],
                        [-0.9847, -0.6569, -0.1467,  1.2373],
                        [ 0.1478, -0.4505,  0.1463,  0.1611]])

In [7]:
%timeit t.a = new_value

78.3 ns ± 0.455 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.0058,  0.4760, -0.0354],
               [ 0.6147,  1.4466, -0.1422]]),
    x: Batch(
           c: tensor([[-0.5109,  0.0742, -0.8903, -0.1564],
                      [-0.9847, -0.6569, -0.1467,  1.2373],
                      [ 0.1478, -0.4505,  0.1463,  0.1611]]),
       ),
)

In [10]:
b.a

tensor([[ 1.0058,  0.4760, -0.0354],
        [ 0.6147,  1.4466, -0.1422]])

In [11]:
%timeit b.a

70.3 ns ± 0.302 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.4985,  0.0311,  2.0596],
               [ 0.6472,  0.5762, -1.0893]]),
    x: Batch(
           c: tensor([[-0.5109,  0.0742, -0.8903, -0.1564],
                      [-0.9847, -0.6569, -0.1467,  1.2373],
                      [ 0.1478, -0.4505,  0.1463,  0.1611]]),
       ),
)

In [13]:
%timeit b.a = new_value

601 ns ± 2.32 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

982 ns ± 2.86 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.9 µs ± 84.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

168 µs ± 924 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

166 µs ± 2.31 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fc4dee510d0>
├── 'a' --> tensor([[[ 1.0058,  0.4760, -0.0354],
│                    [ 0.6147,  1.4466, -0.1422]],
│           
│                   [[ 1.0058,  0.4760, -0.0354],
│                    [ 0.6147,  1.4466, -0.1422]],
│           
│                   [[ 1.0058,  0.4760, -0.0354],
│                    [ 0.6147,  1.4466, -0.1422]],
│           
│                   [[ 1.0058,  0.4760, -0.0354],
│                    [ 0.6147,  1.4466, -0.1422]],
│           
│                   [[ 1.0058,  0.4760, -0.0354],
│                    [ 0.6147,  1.4466, -0.1422]],
│           
│                   [[ 1.0058,  0.4760, -0.0354],
│                    [ 0.6147,  1.4466, -0.1422]],
│           
│                   [[ 1.0058,  0.4760, -0.0354],
│                    [ 0.6147,  1.4466, -0.1422]],
│           
│                   [[ 1.0058,  0.4760, -0.0354],
│                    [ 0.6147,  1.4466, -0.1422]]])
└── 'x' --> <FastTreeValue 0x7fc4dee51af0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

37 µs ± 214 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fc4dee6af40>
├── 'a' --> tensor([[ 1.0058,  0.4760, -0.0354],
│                   [ 0.6147,  1.4466, -0.1422],
│                   [ 1.0058,  0.4760, -0.0354],
│                   [ 0.6147,  1.4466, -0.1422],
│                   [ 1.0058,  0.4760, -0.0354],
│                   [ 0.6147,  1.4466, -0.1422],
│                   [ 1.0058,  0.4760, -0.0354],
│                   [ 0.6147,  1.4466, -0.1422],
│                   [ 1.0058,  0.4760, -0.0354],
│                   [ 0.6147,  1.4466, -0.1422],
│                   [ 1.0058,  0.4760, -0.0354],
│                   [ 0.6147,  1.4466, -0.1422],
│                   [ 1.0058,  0.4760, -0.0354],
│                   [ 0.6147,  1.4466, -0.1422],
│                   [ 1.0058,  0.4760, -0.0354],
│                   [ 0.6147,  1.4466, -0.1422]])
└── 'x' --> <FastTreeValue 0x7fc560a81e80>
    └── 'c' --> tensor([[-0.5109,  0.0742, -0.8903, -0.1564],
                        [-0.9847, -0.6569, -0.1467,  1.2373],
                 

In [23]:
%timeit t_cat(trees)

34.8 µs ± 238 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

65.3 µs ± 395 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.0058,  0.4760, -0.0354],
                [ 0.6147,  1.4466, -0.1422]],
       
               [[ 1.0058,  0.4760, -0.0354],
                [ 0.6147,  1.4466, -0.1422]],
       
               [[ 1.0058,  0.4760, -0.0354],
                [ 0.6147,  1.4466, -0.1422]],
       
               [[ 1.0058,  0.4760, -0.0354],
                [ 0.6147,  1.4466, -0.1422]],
       
               [[ 1.0058,  0.4760, -0.0354],
                [ 0.6147,  1.4466, -0.1422]],
       
               [[ 1.0058,  0.4760, -0.0354],
                [ 0.6147,  1.4466, -0.1422]],
       
               [[ 1.0058,  0.4760, -0.0354],
                [ 0.6147,  1.4466, -0.1422]],
       
               [[ 1.0058,  0.4760, -0.0354],
                [ 0.6147,  1.4466, -0.1422]]]),
    x: Batch(
           c: tensor([[[-0.5109,  0.0742, -0.8903, -0.1564],
                       [-0.9847, -0.6569, -0.1467,  1.2373],
                       [ 0.1478, -0.4505,  0.1463,  0.1611]],
         

In [26]:
%timeit Batch.stack(batches)

90.5 µs ± 567 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.0058,  0.4760, -0.0354],
               [ 0.6147,  1.4466, -0.1422],
               [ 1.0058,  0.4760, -0.0354],
               [ 0.6147,  1.4466, -0.1422],
               [ 1.0058,  0.4760, -0.0354],
               [ 0.6147,  1.4466, -0.1422],
               [ 1.0058,  0.4760, -0.0354],
               [ 0.6147,  1.4466, -0.1422],
               [ 1.0058,  0.4760, -0.0354],
               [ 0.6147,  1.4466, -0.1422],
               [ 1.0058,  0.4760, -0.0354],
               [ 0.6147,  1.4466, -0.1422],
               [ 1.0058,  0.4760, -0.0354],
               [ 0.6147,  1.4466, -0.1422],
               [ 1.0058,  0.4760, -0.0354],
               [ 0.6147,  1.4466, -0.1422]]),
    x: Batch(
           c: tensor([[-0.5109,  0.0742, -0.8903, -0.1564],
                      [-0.9847, -0.6569, -0.1467,  1.2373],
                      [ 0.1478, -0.4505,  0.1463,  0.1611],
                      [-0.5109,  0.0742, -0.8903, -0.1564],
                      [-0.9847, -

In [28]:
%timeit Batch.cat(batches)

167 µs ± 725 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

369 µs ± 1.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
